# LDA에서 Dynamic Topic Model

- LDA를 time slice 단위로 여러 개를 붙인 형태
- 토픽 분포에 해당하는 파라미터인 Beta 값을 time slice가 넘어갈 때 넘겨주는 것으로 시간대별 토픽 분포를 반영하게 함.

In [1]:
import pickle
from pprint import pprint
import pandas as pd
import numpy as np
import re

In [2]:
# 데이터 불러오기
with open("./data/cleaned_data.pk", "rb") as f:
    data = pickle.load(f)

data.reset_index(inplace=True, drop=True)
print(data.info())
print(data.tail())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2223 entries, 0 to 2222
Data columns (total 3 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Date     2223 non-null   object
 1   User     2223 non-null   object
 2   Message  2223 non-null   object
dtypes: object(3)
memory usage: 52.2+ KB
None
                     Date          User                   Message
2218  2021-03-30 10:19:34    Rt회계학과 양윤철  내친구 댕기는 회사는 전액 회사 부담이라서 
2219  2021-03-30 10:19:41  Rt글로벌비즈니스최성진                         굿
2220  2021-03-30 10:19:42  Rt글로벌비즈니스최성진                       복지굿
2221  2021-03-30 10:19:54  Rt글로벌비즈니스최성진                  너가 건의해봐 
2222  2021-03-30 10:20:29  Rt글로벌비즈니스최성진                   장우용 핼맷컷


### 분석시기 설정하기

In [3]:
### Data column을 Pandas Datetime으로 변환
data["Date"] = pd.to_datetime(data["Date"])
# Date Column을 index로 설정
data = data.set_index("Date")
data.tail()

,User,Message
Date,,
2021-03-30 10:19:34,Rt회계학과 양윤철,내친구 댕기는 회사는 전액 회사 부담이라서
2021-03-30 10:19:41,Rt글로벌비즈니스최성진,굿
2021-03-30 10:19:42,Rt글로벌비즈니스최성진,복지굿
2021-03-30 10:19:54,Rt글로벌비즈니스최성진,너가 건의해봐
2021-03-30 10:20:29,Rt글로벌비즈니스최성진,장우용 핼맷컷


### 월별 대화내용 분리(DTM 분석을 위해)

In [4]:
# 월 별로 나누기
month02 = data["2021-02-01" : "2021-02-28"]
month03 = data["2021-03-01" : "2021-03-30"]
print(month02.tail())
print(month03.tail())

                          User        Message
Date                                         
2021-02-28 17:08:41  Rt건축공학조수영       사업하고싶은사람
2021-02-28 17:08:53  Rt건축공학조수영  50기 선배가 물어보더라
2021-02-28 17:08:56  Rt건축공학조수영       아마 카페일텐데
2021-02-28 17:17:48    Rt법학차진영          오 조수영
2021-02-28 17:17:52    Rt법학차진영      활발하게 활동하네
                             User                   Message
Date                                                       
2021-03-30 10:19:34    Rt회계학과 양윤철  내친구 댕기는 회사는 전액 회사 부담이라서 
2021-03-30 10:19:41  Rt글로벌비즈니스최성진                         굿
2021-03-30 10:19:42  Rt글로벌비즈니스최성진                       복지굿
2021-03-30 10:19:54  Rt글로벌비즈니스최성진                  너가 건의해봐 
2021-03-30 10:20:29  Rt글로벌비즈니스최성진                   장우용 핼맷컷


### 시기별로 나누어 데이터 저장하기

In [5]:
# time_slice 별로 데이터 갯수를 저장할 리스트 생성
time_slice = []

# 시기 별로 데이터 가져오기
slice0 = list(month02["Message"])
slice1 = list(month03["Message"])

# LDA 분석할 때와 비슷하게 모든 분석 대상 텍스트를 담는 리스트 생성
tokenized_data = [msg.split() for msg in (slice0 + slice1)]

# 각 slice에 들어있는 갯수를 원소로 갖는 리스트 생성
time_slice.append(len(slice0))
time_slice.append(len(slice1))

In [8]:
print(len(tokenized_data))
print(time_slice)
print(slice0[6:11])
print(slice1[6:11])
pprint(tokenized_data[5:10])

2223
[97, 2126]
['뭐', '뭘그만해', '칭찬해주는걸 멈추라는건', '내겐 가장어렵지', '넷마블 어제보다 7천원올랐다']
['샵검색  쿨', '오후', '진영이 생일축하해', '샵검색  쿨타임', '진영이 이방에 없음']
[['그만해라'], ['뭐'], ['뭘그만해'], ['칭찬해주는걸', '멈추라는건'], ['내겐', '가장어렵지']]


### Dynamic Topic Model 돌리기

In [9]:
from gensim.models import ldaseqmodel
from gensim.corpora import Dictionary, bleicorpus
from gensim import corpora

import os

/Users/hyunsoolee/opt/anaconda3/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


In [10]:
# DTM dictionary 저장.
if not os.path.exists("kakao(DTM)_dict"):
    dictionary = corpora.Dictionary(tokenized_data)
    dictionary.save("kakao(DTM)_dict")
else:
    dictionary = Dictionary.load("kakao(DTM)_dict")
    
# DTM Corpus 저장.
if not os.path.exists("kakao(DTM)_corpus"):
    corpus = [dictionary.doc2bow(doc) for doc in tokenized_data]
    corpora.BleiCorpus.serialize("kakao(DTM)_corpus", corpus)
else:
    corpus = bleicorpus.BleiCorpus("kakao(DTM)_corpus")

### RUN DTM MODEL

In [12]:
NUM_TOPICS=4

if not os.path.exists("kakao(DTM)_model"):
    dtm_model = ldaseqmodel.LdaSeqModel(corpus=corpus, id2word=dictionary, time_slice=time_slice,
                                       num_topics=NUM_TOPICS, passes=5)
    dtm_model.save("kakao(DTM)_model")
else:
    dtm_model = ldaseqmodel.LdaSeqModel.load("kakao(DTM)_model")

/Users/hyunsoolee/opt/anaconda3/lib/python3.8/site-packages/gensim/models/ldaseqmodel.py:1478: RuntimeWarning: invalid value encountered in double_scalars
  converged = np.fabs((lhood_old - lhood) / (lhood_old * total))
/Users/hyunsoolee/opt/anaconda3/lib/python3.8/site-packages/gensim/models/ldaseqmodel.py:297: RuntimeWarning: divide by zero encountered in double_scalars
  convergence = np.fabs((bound - old_bound) / old_bound)
/Users/hyunsoolee/opt/anaconda3/lib/python3.8/site-packages/gensim/models/ldaseqmodel.py:1478: RuntimeWarning: invalid value encountered in double_scalars
  converged = np.fabs((lhood_old - lhood) / (lhood_old * total))
/Users/hyunsoolee/opt/anaconda3/lib/python3.8/site-packages/gensim/models/ldaseqmodel.py:1478: RuntimeWarning: invalid value encountered in double_scalars
  converged = np.fabs((lhood_old - lhood) / (lhood_old * total))
/Users/hyunsoolee/opt/anaconda3/lib/python3.8/site-packages/gensim/models/ldaseqmodel.py:1478: RuntimeWarning: invalid value enc

### 결과보기

In [19]:
# 고정된 시간 내에서 전체 토픽 보기
pprint(dtm_model.print_topics(time=0, top_terms=2))

[[('사진', 0.05438203109051702), ('메시지입니다', 0.05409506803932079)],
 [('굿', 0.021871439977078792), ('와', 0.015117218474390078)],
 [('오', 0.020338860799794183), ('와우', 0.012608191243409325)],
 [('아', 0.03748283104539533), ('진짜', 0.015728025958138533)]]
